# Coleta de Dados

Vamos começar importando as bibliotecas que usaremos para obter os dados.

In [1]:
import bs4
import pandas as pd
import requests
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

Agora vamos acessar a página da Wikipedia dos municípios de São Paulo usando a biblioteca BeautifulSoup4 e extrair o nome e o link da página da Wikipedia de cada cidade.

In [2]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_municípios_de_São_Paulo'
page = requests.get(url)
soup = bs4.BeautifulSoup(page.text, "html.parser")

In [3]:
municipios = []
links = []
for x in soup.find('table', class_='wikitable sortable').find_all(style='text-align:left;'):
    municipios.append(x.find_all('a')[-1].get_text())
    mun_url = requests.get('https://pt.wikipedia.org' + x.find_all('a')[-1]['href'])
    mun_soup = bs4.BeautifulSoup(mun_url.text, 'html.parser')
    redirected_url = mun_soup.find('link', rel='canonical')['href']
    links.append(redirected_url)

Com os links de cada cidade, extrairemos os dados referentes aos municípios de São Paulo que são limítrofes de cada munícipio de São Paulo.

In [4]:
municipios_limitrofes = []
for i, l in enumerate(links):
    municipios_limitrofes.append([])
    page_l = requests.get(l)
    soup = bs4.BeautifulSoup(page_l.text, "html.parser")
    soup_td = soup.find('table', class_='infobox_v2').find_all('td')
    for j, limit in enumerate(soup_td):
        if limit.get_text().replace('\n', '').strip() == 'Municípios limítrofes':
            for mun in soup_td[j+1].find_all('a'):
                mun_url = requests.get('https://pt.wikipedia.org' + mun['href'])
                mun_soup = bs4.BeautifulSoup(mun_url.text, 'html.parser')
                redirected_url = mun_soup.find('link', rel='canonical')['href']
                if redirected_url in links:
                    municipios_limitrofes[i].append(municipios[links.index(redirected_url)])    

Se um município x é vizinho de um município y, esse município y também é vizinho do município x. Portanto, vamos checar se não há nenhuma cidade faltando na lista de cidades limítrofes de cada cidade e, caso haja, adicioná-la à lista.

In [5]:
for i, c in enumerate(municipios_limitrofes):
    for j, m in enumerate(municipios_limitrofes):
        if municipios[i] in m and municipios[j] not in c:
            municipios_limitrofes[i].append(municipios[j])

Agora vamos salvar a lista de municípios e seus municípios limítrofes em um arquivo TXT para caso precisemos acessar os dados nvoamente.

In [6]:
with open('municipios_sp.txt', 'a') as m:
    for mun in municipios:
        m.write(mun + '\n')

In [7]:
with open('municipios_sp_limitrofes.txt', 'a') as ml:
    for mun_limit in municipios_limitrofes:
        ml.write(str(mun_limit) + '\n')

Para facilitar o acesso das listas, vamos criar um dicionário que mapeia os nomes para os respectivos índices.

In [8]:
name_to_idx = dict()
for i in range(len(municipios)):
    name_to_idx[municipios[i]] = i

Agora podemos começar a coletar os dados referente às distâncias entre cada município vizinho. Primeiramente, vamos obter a latitude e longitude de cada cidade acesando a API do Nominatim por meio do GeoPy.

In [9]:
geolocator = Nominatim(user_agent='was5@cin.ufpe.br')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)
lat_long = [list(geocode({'city': municipio, 'state': 'São Paulo', 'country': 'Brasil'}))[-1] for municipio in municipios]

Também salvaremos esses dados em um arquivo TXT.

In [10]:
with open('lat_long_municipios_sp.txt', 'a') as ll:
    for latlong in lat_long:
        ll.write(str(latlong)+'\n')

Agora que temos as coordenadas de cada cidade, vamos utilizar a API do Open Street Routing Machine (OSRM) para obter os dados da distância de carro entre as cidades vizinhas. 
Como faremos uma quantidade razoável de acessos, vamos acessar a API localmente usando o Docker. Para isso, vamos baixar [os dados do OpenStreetMap da região sudeste do Brasil](http://download.geofabrik.de/south-america/brazil/sudeste.html) e proceder conforme consta no [repositório da OSRM](https://github.com/Project-OSRM/osrm-backend).

Para informações sobre o funcionamento da API, acesse a [documentação da OSRM](http://project-osrm.org/docs/v5.23.0/api/#).

In [11]:
arestas = []
estradas = []
for i, mun_limits in enumerate(municipios_limitrofes):
    for ml in mun_limits:
        j = name_to_idx[ml]
        if not {i, j} in arestas:
            arestas.append({i, j})
            distancias = []
            for route in requests.get(f'http://localhost:5000/route/v1/car/{lat_long[i][1]},{lat_long[i][0]};{lat_long[j][1]},{lat_long[j][0]}?overview=false&alternatives=3').json()['routes']:
                distancias.append(round(route['distance']/1000, 1))
            estradas.append([i, j, min(distancias)])

Por fim, vamos salvar esses dados em um arquivo TXT.

In [12]:
with open('distancias_municipios_sp.txt', 'a') as dm:
    for e in estradas:
        dm.write(f'{e[0]} {e[1]} {e[2]}\n')